In [17]:
#import from regular python

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb

cvr_Path = "/home/svanhmic/workspace/DABAI/ReadData" # TODB alter path!

In [18]:
#imports from Pyspark

from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans

sc.addPyFile(cvr_Path+"/ConvertAllToVecToMl.py")
import ConvertAllToVecToMl # included homebrewed transformation for converting input vector to double vector. 

In [14]:
data_Pdf = sb.load_dataset("flights") # naive data set from seaborn

In [19]:
#create spark data frame

In [20]:
#create the spark ml pipeline

In [21]:
#execute the pipeline

In [22]:
#return the data to Pandas for visualisation